In [1]:
import torch
from torchvision import transforms, models
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import argparse
import os
from datetime import datetime
import shutil
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import cv2

from utils_logging import setup_logger

In [2]:
from models.shashimal2 import Shashimal2
from models.__init__ import save_checkpoint, resume_checkpoint
#from dataloader.shashimal2_synth import GooDataset
from dataloader.shashimal2 import GooDataset
from dataloader import chong_imutils
from training.train_shashimal2 import train, test, GazeOptimizer

In [3]:
logger = setup_logger(name='first_logger',
                      log_dir ='./logs/',
                      log_file='train_chong_gooreal.log',
                      log_format = '%(asctime)s %(levelname)s %(message)s',
                      verbose=True)

In [4]:
batch_size=4
workers=4

images_dir = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/Datasets/gooreal/finalrealdatasetImgsV2'
pickle_path = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/Datasets/gooreal/oneshotrealhumansNew.pickle'
test_images_dir = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/Datasets/gooreal/finalrealdatasetImgsV2'
test_pickle_path = '/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/Datasets/gooreal/testrealhumansNew.pickle'

train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4)

test_set = GooDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = DataLoader(test_set, batch_size=batch_size//2,
                            shuffle=False, num_workers=4)

Number of Images: 2450
Number of Images: 2146


In [8]:
batch_size=4
workers=4

images_dir = '/media/primesh/F4D0EA80D0EA4906/goosynth_test_v2/images/'
pickle_path = '/media/primesh/F4D0EA80D0EA4906/goosynth_test_v2/data/'
train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4)

In [6]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = Shashimal2()


model_ft = model_ft.to(device)

criterion = nn.NLLLoss().cuda()
# criterion = 

# Observe that all parameters are being optimized
start_epoch = 0
max_epoch = 5
learning_rate = 1e-4

# Initializes Optimizer
gaze_opt = GazeOptimizer(model_ft, learning_rate)
optimizer = gaze_opt.getOptimizer(start_epoch)
if False:
    checkpoint_fpath = './saved_models/shashimal2/chechkpoint1.pt'
    checkpoint = torch.load(checkpoint_fpath)
    model_ft.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

26


**Model graph**

In [10]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/chong_spatial')

# get some random training images
dataiter = iter(train_data_loader)
img, face, head_channel, object_channel,gaze_heatmap, image_path, gaze_inside,shifted_targets,gaze_final = dataiter.next()

writer.add_graph(model_ft, (img.cuda(), face.cuda(), head_channel.cuda(), object_channel.cuda()))
writer.close()

/home/primesh/anaconda3/envs/FYP/lib/python3.9/site-packages/torch/jit/_trace.py:934: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  module._c._create_method_from_trace(


In [8]:
img, face, head_channel, object_channel,gaze_heatmap, image_path, gaze_inside,shifted_targets,gaze_final = next(iter(train_data_loader))

In [9]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/shashimal2_pretrained')
#writer.add_graph(model_ft, (img.cuda(),face.cuda(),head_channel.cuda(),object_channel.cuda()))
# writer.close()


In [12]:
model_ft = train(model_ft,train_data_loader, criterion, optimizer, logger, writer,num_epochs=4)

  0%|          | 0/4800 [00:04<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.95 GiB total capacity; 926.04 MiB already allocated; 11.19 MiB free; 968.00 MiB reserved in total by PyTorch)

In [ ]:
torch.save(model_ft.state_dict(), 'model_ft.pth')

In [18]:
%load_ext tensorboard
%tensorboard --logdir="/media/primesh/F4D0EA80D0EA4906/PROJECTS/FYP/Gaze detection/code/ObjectGazeNet/gazefollowing/runs" 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [13]:
state = {'epoch': 2, 'state_dict': model_ft.state_dict(),
             'optimizer': optimizer.state_dict(), 'logger': logger, }
filename = './saved_models/shashimal2/chechkpoint1.pt'
torch.save(state, filename)

In [10]:
torch.save(model_ft.state_dict(), 'shashimal2_synth_1.pth')

In [12]:
shashimal_model = Shashimal2()
shashimal_model.load_state_dict(torch.load('shashimal2_synth_1.pth'))
shashimal_model.cuda()

26


Shashimal2(
  (relu): ReLU(inplace=True)
  (sigmoid): Sigmoid()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=7, stride=1, padding=0)
  (conv1_scene): Conv2d(5, 3, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn1_scene): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (compress_conv0): Conv2d(4096, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (compress_bn0): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (compress_conv1): Conv2d(2048, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (compress_bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (compress_conv2): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (compress_bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (compress_conv3): Conv2d(512, 256, kernel_size=(1, 1), s

In [31]:
import datetime

In [32]:
img, face, head_channel, object_channel, eyes_loc, gaze_heatmap, gaze, gaze_inside, image_path, gaze_final, gtbox,eye =  next(iter(test_data_loader))

a = datetime.datetime.now()
outputs, raw_hm = shashimal_model.raw_hm(img.cuda(),face.cuda(),head_channel.cuda(),object_channel.cuda())
b = datetime.datetime.now()
c = b - a
c.microseconds/1000000


0.823196

In [21]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
idxs = raw_hm[0].detach().cpu().numpy()
idxs.shape
plt.imshow(idxs, cmap='hot', interpolation='nearest')
plt.savefig("hm.png")
idxs = img[0].detach().cpu().numpy()
idxs = idxs.swapaxes(0,1)
idxs = idxs.swapaxes(1,2)
idxs.shape
plt.imshow(idxs)
plt.savefig("img.png")

<IPython.core.display.Javascript object>

In [ ]:
import torch.nn.utils.prune as prune
for name, module in shashimal_model.named_modules():
    # prune 20% of connections in all 2D-conv layers 
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    # prune 40% of connections in all linear layers 
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

print(dict(shashimal_model.named_buffers()).keys())

In [29]:
a = datetime.datetime.now()
outputs, raw_hm = shashimal_model.raw_hm(img.cuda(),face.cuda(),head_channel.cuda(),object_channel.cuda())
b = datetime.datetime.now()
c = b - a
c.microseconds/1000000


0.575163

In [24]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
idxs = raw_hm[0].detach().cpu().numpy()
idxs.shape
plt.imshow(idxs, cmap='hot', interpolation='nearest')
plt.savefig("hm_prune.png")
idxs = img[0].detach().cpu().numpy()
idxs = idxs.swapaxes(0,1)
idxs = idxs.swapaxes(1,2)
idxs.shape
plt.imshow(idxs)
plt.savefig("img_prune.png")

<IPython.core.display.Javascript object>

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
idxs = raw_hm[1].detach().cpu().numpy()
idxs.shape
plt.imshow(idxs, cmap='hot', interpolation='nearest')
plt.savefig("hm1.png")
idxs = img[1].detach().cpu().numpy()
idxs = idxs.swapaxes(0,1)
idxs = idxs.swapaxes(1,2)
idxs.shape
plt.imshow(idxs)
plt.savefig("img1.png")

In [13]:
test(shashimal_model, test_data_loader, logger, save_output=True)

100%|██████████| 1073/1073 [10:34<00:00,  1.69it/s]
proximate accuracy: [0.00326188 0.02749301 0.07595527 0.24510718 0.4333644  0.59972041
 0.722274   0.82059646]
average error: [0.8838075344160241, 0.1923638245821395, 32.7021577874671]


[0.8838075344160241, 0.1923638245821395, 32.7021577874671]

In [10]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
kernels = shashimal_model.compress_conv0.weight.detach().clone()
kernels = kernels - kernels.min()
kernels = kernels / kernels.max()
img = make_grid(kernels)
plt.imshow(img.permute(1, 2, 0))

TypeError: Invalid shape (770, 26, 4096) for image data

In [ ]:
m = nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3, dilation=1, groups=1, bias=True)
p = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
input = torch.randn(4, 3, 448, 448)
output = p(m(input))
print(input.shape)
print(output.shape)

torch.Size([4, 3, 448, 448])
torch.Size([4, 64, 448, 448])
